### Description of packages to run

1. <b>bs4</b> has the BeautifulSoup package that is used for web scraping <br>
2. <b>requests</b> garners information from a particular website <br>
3. <b>time</b> is used to ascertain the time the code is running for a particular link <br>
4. <b>sleep</b> suspends (waits) execution of the current thread for a given number of seconds <br>
5. <b>sys</b> is used for exception handling <br>
6. <b>Urllib</b> package is the URL handling module for python. It is used to fetch URLs (Uniform Resource Locators) it uses the <b>urlopen</b> function and is able to fetch URLs using a variety of different protocols <br>

In [ ]:
import bs4 as bs
from bs4 import BeautifulSoup
import requests
import time
from time import sleep
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from random import randint
import re
from urllib.request import urlopen

<b> Web scraping Intro </b> <br>

Webscraping involves extraction information from the html texts, the specifics of which are embedded in certain classes that come under some tags. For instance 'div' is a tag, under 'div' lies the 'class' = 'author date' which will give us the date when the article was published <br>

<b> Exception Handling </b> <br>

Sometimes there can be errors such as 'NoneType' object has no attribute 'text', to avoid such errors to stop the code 
to execute completely, try and except function is used. The code to be run within each url is embedded under 'try' and if
there are any exceptions then it passes through 'except', which bypasses the error to contine the code execution. <br>

<b> WARNING!!! </b> <br>
VinylPulse is an exceptionally difficult website to scrape, it has issues with loading urls properly, may load on the 
phone but may not on the laptop. It also can detect webscraping effectively, hence to avoid getting blocked:<br>
1. One must enter a code to mimic human behaviour, sleep ensures that the code stays on a particular url, the time is randomised using the 'randint' function. Time is in seconds. <br>
2. Run the code chunk below for maximum 20 pages at a time. <br>
3. If VPN is available, change the region after 2-3 tries <br>

In [ ]:
#Code to append details in the dataframe (mainly to garner the links for each collectible), with using try 
#and except to bypass any exceptions

page = 1
title = []
category = []
author = []
date = []
actions = []
more_info = []

while page != 50:
    url = f"https://www.vinylpulse.com/category/incoming/page/{page}"
    html_text = requests.get(url).text
    sleep(randint(60,120))
    soup = BeautifulSoup(html_text,'html', multi_valued_attributes = None)
    collectibles = soup.find_all('article')
    
    try:
        
        for collectible in collectibles:
            title.append(collectible.find('h2', class_ = 'entry-title').text)
            author.append(collectible.find('div', class_ = 'author date').text.replace('By:',''))
            date.append(collectible.find('div', {"class": "date"}).text)
            actions.append(collectible.find('div', class_ = 'actions').text)
            more_info.append(collectible.h2.a['href'])

        page = page + 1
        print(url)
        print(len(collectibles))

    except Exception as e:
        print(e)
        pass

df = pd.DataFrame(list(zip(title, category, author, date, actions, more_info)),
               columns =['Title', 'Category', 'Author', 'Date', 'Social Media', 'Link'])

display(df)

All of these dataframes should be stored in csv to be concatenated later (storing in csv is a better idea in case the extraction occurs over days) <br>

<b> To store a csv file </b> <br> 
df.to_csv(r'C:/*File path*/*name of file*.csv', index=False) <br>

<b> Then read these files </b> <br> 
df = pd.read_csv('C:/*File path*/*name of file*.csv')<br> 
df1 pd.read_csv('C:/*File path*/*name of file*.csv')<br> 

<b> To concatenate the various files </b> <br> 
dataframe = pd.concat([df ,df1, df2, df3, df4, df5, df6, df7, df8]) <br>
dataframe <br>

<b> Data Cleaning </b> <br> 
The collectible name, brand and artist information are not easily scrapable from the site, hence the link is used to extract this information. The trend is the past year is observed that the collectible name is separated by a preposition and the artist and brand name are separated by 'x'. <br>
After all the cleaning on python, a manual check should be done because this may not always render the right results

In [ ]:
# Storing split value columns
new = dataframe["Link"].str.split('by|From|With|At|On|For',expand=True)
  
# Making separate last name column from new data frame
dataframe["Collectibles"]= new[0]
dataframe["Artist/Company"]= new[1]

# Cleaning the hypen and html from the 'Artist/Company'
dataframe['Artist/Company'] = dataframe['Artist/Company'].str.replace("-", " ")
dataframe['Artist/Company'] = dataframe['Artist/Company'].str.replace(".html", "")

# Storing split value columns
new1 = dataframe["Artist/Company"].str.split(' x ',expand=True)
  
# Making separate last name column from new data frame
dataframe["Artist"] = new1[0]

dataframe["Company"] = new1[1]

# Display dataframe
dataframe

<b> Extraction from individual link </b> <br>
Each link has the description of the product such as the size, color, edition, material and tags. We need to extract this html text of the description and then use that to find the attributes we are interested in. 

In [ ]:
#Using each link to extract the html text embedded within
text = []
link = []


try:
    for i in list(dataframe['Link'].iloc[1:50]):
        url = i
        x = urlopen(url)
        start = time.time()
        new = x.read()
        soup = bs.BeautifulSoup(new,"lxml", multi_valued_attributes = None)
        text.append(soup.find_all('p'))
        sleep(randint(90,150))
        end = time.time()
        elapsed = end - start
        print(str(elapsed))
        link.append(i)

except Exception as e:
    print(e)
    pass
    
vinyl = {'Text': text, 'Link': link}
dataframe_text = pd.DataFrame(vinyl)
display(dataframe_text) 

<b> Defining price and size in a function using regex to extract information from text </b> <br>

Price starts with a dollar sign can be whole numbers, decimals, separated by a comma or a combination of both eg <br> 

1. 185.93 - https://www.vinylpulse.com/2021/07/kid-katana-005-renegade-leader-by-2petalrose.html <br>
2. 3,850 - https://www.vinylpulse.com/2021/06/mickey-mouse-mosaic-art-style-by-disney-x-milk.html <br>  
3. 135 - https://www.vinylpulse.com/2021/06/it-bear-bob-v2-by-milkboy-toys.html <br>

Regex breakdown:\$[0-9]*[,]*[0-9]*[.]*[0-9]{2} <br>
- \$ - in regex '$' is a special character, hence to nullify that, in the code it is preceeded by a \ <br>
- [0-9]* - any number of digits, * signifying that zero or more instances <br>
- [,]*[0-9]* - digits can/cannot be followed by a comma (hence *), which can be followed by some digits if comma is present <br>
- [.]*[0-9]{2} - post the decimal, if there is (hence *), two additional digits must be there (trend in VP) <br>

Size on the website is followed by the inch sign ″ and might be a decimal <br>

Regex breakdown: \d*?\.?\d*″ <br>
- \d*?\.?\d*″ - any number of digits, optional (hence ?) <br>
- \.? - decimal point is optional <br>
- \d*″ - any number of digits followed by ″ <br>

In [ ]:
def price(Text):
    text1 = str(Text)
    price = re.findall(r'\$[0-9]*[,]*[0-9]*[.]*[0-9]{2}', text1)
    return price

def size(Text):
    text2 = str(Text)
    size =  re.findall(r'\d*?\.?\d*″', text2)
    return size

In [ ]:
# Applying function to the dataframe
dataframe_text['Price'] = dataframe_text['Text'].apply(price)
dataframe_text['Size'] = dataframe_text['Text'].apply(size)

Data cleaning for price and size column as the apply function renders a list that is not usable for data analysis

In [ ]:
dataframe_text['Price'] = dataframe_text['Price'].astype(str).str.replace(r'\[|\]', '')
dataframe_text['Price'] = dataframe_text['Price'].str.replace(',', '')
dataframe_text['Price'] = dataframe_text['Price'].str.replace("$", "")
dataframe_text['Price'] = dataframe_text['Price'].str.replace("'", "")

dataframe_text['Size'] = dataframe_text['Size'].astype(str).str.replace(r'\[|\]', '')
dataframe_text['Size'] = dataframe_text['Size'].astype(str).str.replace(r'″', '')
dataframe_text['Size'] = dataframe_text['Size'].astype(str).str.replace(r"'", '')


<b> Manual Data Cleaning </b> <br>
The above code removes all the unneccesary characters, however a manual clean up is required in case there are instances where there are multiple prices and sizes, a call needs to be made whether to duplicate the row or to eliminate the row with the smaller/larger price, smaller/larger size. After the manual clean up, the price and size column need to be converted to float type to enable data analysis

In [ ]:
dataframe_text['Price'] = pd.to_numeric(dataframe_text['Price'])
dataframe_text['Size'] = pd.to_numeric(dataframe_text['Size'])

Tags are the important key words for each link which start with 'tag' with text embedded between <a?> 
The following code identifies the entire block within tag, then splits it with a comma, filters out the html link and 'tag' and appends them under the column 'Tags' 

In [ ]:
#locate tags
textlist = dataframe_text['Text'].str.findall(r'rel="tag">.*</a>').str.join(", ").fillna('')

#remove html tags and links
tags = []
innertags = []
textlistsplit =[x.split(',') for x in textlist]
for i in textlistsplit:
    for j in i:
        j = re.sub(r'rel="tag">', '', j)
        j = re.sub(r'</a>', '' ,j)
        j = re.sub(r'<a href="http\S+', '', j)
        j = j.lstrip()
        innertags.append(j)
    tags.append(innertags)
    innertags = []

add tags as new column
dataframe_text['Tags'] = tags

In [ ]:
# Data cleaning for tags
dataframe_text['Tags'] = dataframe_text['Tags'].astype(str).str.replace(r'\[|\]', '')
dataframe_text['Tags'] = dataframe_text['Tags'].astype(str).str.replace(r"'", '')

In [ ]:
#Combine all of these links using concat in panda and then 'Link' will be used iteratively to extract price
#size and tagged information

dataframe_link = pd.concat([dataframe_text ,dataframe_text1, dataframe_text2, dataframe_text3, dataframe_text4])
dataframe_link

#Save this as a csv file

In [ ]:
# Merge the two main csv files by 'link'

#merge
df_comp = pd.merge(dataframe, dataframe_link, on = 'Link')
df_comp

#### Basic Data Analysis

In [ ]:
df_comp.columns

In [ ]:
df_comp.isnull().sum()

In [ ]:
#Taking columns where 'Price' is not NA

df_comp = df_comp[frame['Price'].notna()]

# to verify
df_comp.isnull().sum()

In [ ]:
# Unique companies in the dataset
print(df_comp['Company'].unique())

In [ ]:
# Count of companies, then finding the top 11
frequency_co = frame['Company'].value_counts().nlargest(11)
frequency_co

In [ ]:
# Compiling the list of top 11 companies
comp_name = ['Unbox Industries','Mighty Jaxx','Martian Toys','Uvd Toys', 'Clutter','Superplastic','Fools Paradise','3Dretro','Kidrobot','Allrightsreserved','Sank Toys']

In [ ]:
# Finding the average price from the list of competitors
comp_mean = frame[frame.Company.isin(comp_name)].groupby('Company')['Price'].mean().nlargest(11)
print(comp_mean)

In [ ]:
# Finding the max price from the list of competitors
comp_mean = frame[frame.Company.isin(comp_name)].groupby('Company')['Price'].max().nlargest(11)
print(comp_mean)

In [ ]:
# Finding the min price from the list of competitors
comp_mean = frame[frame.Company.isin(comp_name)].groupby('Company')['Price'].min().nlargest(11)
print(comp_mean)

In [ ]:
# Count of top 20 artists
frequency_art = frame['Artist'].value_counts().nlargest(20)
frequency_art